# BigQuery basics

[BigQuery](https://cloud.google.com/bigquery/docs/) is a petabyte-scale analytics data warehouse that you can use to run SQL queries over vast amounts of data in near realtime. This page shows you how to get started with the Google BigQuery API using the Python client library.

In [1]:
from google.cloud import bigquery
import pandas as pd

## Initialize a client

To use the BigQuery Python client library, start by initializing a client. The BigQuery client is used to send and receive messages from the BigQuery API.

### Client project
The `bigquery.Client` object uses your default project. Alternatively, you can specify a project in the `Client` constructor. For more information about how the default project is determined, see the [google-auth documentation](https://google-auth.readthedocs.io/en/latest/reference/google.auth.html).


### Client location
Locations are required for certain BigQuery operations such as creating a dataset. If a location is provided to the client when it is initialized, it will be the default location for jobs, datasets, and tables.

Run the following to create a client with your default project:

In [2]:
#client = bigquery.Client(location="US")
#print("Client creating using default project: {}".format(client.project))
client = bigquery.Client(location="US", project="prj-p-au-c360-anlytx")

To explicitly specify a project when constructing the client, set the `project` parameter:

 !pip3 install openpyxl

## Importing Libraries - 

In [3]:
%%time
import datetime as dt
from dateutil.relativedelta import relativedelta
import pandas as pd
import os
import sys
import glob
os.environ['SPARK_HOME']='/opt/mapr/spark/spark'
os.environ['PYSPARK_PYTHON']='/opt/python/python37/bin/python'
from pyspark.sql import SparkSession
import pyspark
import getpass
import logging
import numpy as np
import calendar 
from datetime import datetime
from dateutil.tz import gettz
from pyspark.sql.functions import *
from pyspark.sql.types import *
import re
import json
from functools import partial
import requests
import jellyfish

CPU times: user 17.5 ms, sys: 12.8 ms, total: 30.3 ms
Wall time: 28.6 ms


In [4]:
def get_user_choice(options):
    """
    Prompts the user to choose from a list of predefined options or enter a custom value.
    Args:
        options (list): A list of predefined string options.
    Returns:
        str: The user's chosen string.
    """
    num_options = len(options)
    print("\nPlease choose from the following options:")
    for i, option in enumerate(options):
        print(f"{i + 1}. {option}")
    print(f"{num_options + 1}. Enter a custom value")
    while True:
        try:
            choice_str = input(f"Enter your choice (1-{num_options + 1}): ").strip()
            if not choice_str:
                print("Input cannot be empty. Please try again.")
                continue
            choice = int(choice_str)
            if 1 <= choice <= num_options:
                return options[choice - 1]  # Return the chosen predefined option
            elif choice == num_options + 1:
                custom_value = input("Please enter your custom value: ").strip()
                if not custom_value:
                    print("Custom value cannot be empty. Please try again.")
                    continue
                return custom_value  # Return the manually entered value
            else:
                print(f"Invalid choice. Please enter a number between 1 and {num_options + 1}.")
        except ValueError:
            print("Invalid input. Please enter a number.")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

## Please Provide User Input for Data Pull-

In [5]:
# Define your three options
predefined_options = ["cm15", "iclic_id", "oblgr_id"]
# Get the user's choice
selected_value = get_user_choice(predefined_options)

while True:
    try:
        number_str = input("Enter the number of inputs (must be less than 11): ").strip()
        if not number_str:
            print("Input cannot be empty. Please try again.")
            continue # Go back to the start of the loop
        number = int(number_str) # Attempt to convert to integer
        if number < 11 and number > 0: # Ensure it's positive and less than 11
            break # Exit the loop if valid
        elif number <= 0:
            print("Please enter a positive number of inputs.")
        else: # number is 11 or greater
            print("Please enter a number less than 11.")
    except ValueError:
        print("Invalid input. Please enter a whole number.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
print(f"You need to provide {number} value(s) for the '{selected_value}' field.")
# --- COLLECTING THE INPUT VALUES ---
collected_data = []
for i in range(number):
    value = input(f"Enter value {i+1} for '{selected_value}': ").strip()
    if not value:
        print("Input cannot be empty. Please enter a value.")
        # You might want to re-prompt for this specific value, or just append an empty string
        # For simplicity, this example allows empty strings but you can add more validation
    collected_data.append(value) # Appending the string value
print("\n--- Summary ---")
print(f"You entered {number} input(s) for the field '{selected_value}'.")
print(f"Collected values: {collected_data}")


Please choose from the following options:
1. cm15
2. iclic_id
3. oblgr_id
4. Enter a custom value


Enter your choice (1-4):  2
Enter the number of inputs (must be less than 11):  2


You need to provide 2 value(s) for the 'iclic_id' field.


Enter value 1 for 'iclic_id':  000236606885406
Enter value 2 for 'iclic_id':  000268732174137



--- Summary ---
You entered 2 input(s) for the field 'iclic_id'.
Collected values: ['000236606885406', '000268732174137']


## Update the Data Pull query if Required-

In [6]:
formatted_list = ",".join(f"'{item}'" if isinstance(item, str) else str(item) for item in collected_data)
print(formatted_list)

query = (f"""select cstone_last_updatetm,
map_id,
`axp-lumi`.dw.decrypt_sde(`axp-lumi`.dw.get_sde_tag('cm11', 'cust360_commercial'), cm11) as cm11,
bu_in,
`axp-lumi`.dw.decrypt_sde(`axp-lumi`.dw.get_sde_tag('cm15', 'cust360_commercial'), cm15) as cm15,
cg_id,
se10,
iclic_id,oblgr_id,oblgr_nm,
mtch_cd,
mtch_cd_mi,
cert_dun_no,
`axp-lumi`.dw.decrypt_sde(`axp-lumi`.dw.get_sde_tag('tax_id', 'cust360_commercial'), tax_id) as tax_id,
mkt_cd,
bus_nm,
json_value(bus_nm,"$[0].source") as nm_source1,
json_value(bus_nm,"$[0].source") as nm_source1,
json_value(bus_nm,"$[0].type") as nm_source_type1,
json_value(bus_nm,"$[0].bus_name") as bus_nm1,
json_value(bus_nm,"$[1].source") as nm_source2,
json_value(bus_nm,"$[1].type") as nm_source_type2,
json_value(bus_nm,"$[1].bus_name") as bus_nm2,
json_value(bus_nm,"$[2].source") as nm_source3,
json_value(bus_nm,"$[2].type") as nm_source_type3,
json_value(bus_nm,"$[2].bus_name") as bus_nm3,
json_value(bus_nm,"$[3].source") as nm_source4,
json_value(bus_nm,"$[3].type") as nm_source_type4,
json_value(bus_nm,"$[3].bus_name") as bus_nm4,
json_value(bus_nm,"$[4].source") as nm_source5,
json_value(bus_nm,"$[4].type") as nm_source_type5,
json_value(bus_nm,"$[4].bus_name") as bus_nm5,
`axp-lumi`.dw.decrypt_sde(`axp-lumi`.dw.get_sde_tag('bus_ad', 'cust360_commercial'), bus_ad) as bus_ad,
`axp-lumi`.dw.decrypt_sde(`axp-lumi`.dw.get_sde_tag('bus_ad', 'cust360_commercial'), bus_full_ad) as bus_full_ad,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[0].source'),'"','') as ad_source1,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[0].type'),'"','') as ad_type1,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[0].full_address'),'"','') as addr_line1,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[0].last_update_dt'),'"','') as ad_last_update_dt1,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[1].source'),'"','') as ad_source2,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[1].type'),'"','') as ad_type2,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[1].full_address'),'"','') as addr_line2,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[1].last_update_dt'),'"','') as ad_last_update_dt2,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[2].source'),'"','') as ad_source3,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[2].type'),'"','') as ad_type3,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[2].full_address'),'"','') as addr_line3,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[2].last_update_dt'),'"','') as ad_last_update_dt3,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[3].source'),'"','') as ad_source4,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[3].type'),'"','') as ad_type4,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[3].full_address'),'"','') as addr_line4,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[3].last_update_dt'),'"','') as ad_last_update_dt4,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[4].source'),'"','') as ad_source5,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[4].type'),'"','') as ad_type5,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[4].full_address'),'"','') as addr_line5,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[4].last_update_dt'),'"','') as ad_last_update_dt5,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[5].source'),'"','') as ad_source6,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[5].type'),'"','') as ad_type6,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[5].full_address'),'"','') as ad_full_address6,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[5].last_update_dt'),'"','') as ad_last_update_dt6,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[6].source'),'"','') as ad_source7,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[6].type'),'"','') as ad_type7,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[6].full_address'),'"','') as ad_full_address7,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[6].last_update_dt'),'"','') as ad_last_update_dt7,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[7].source'),'"','') as ad_source8,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[7].type'),'"','') as ad_type8,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[7].full_address'),'"','') as ad_full_address8,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[7].last_update_dt'),'"','') as ad_last_update_dt8,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[8].source'),'"','') as ad_source9,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[8].type'),'"','') as ad_type9,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[8].full_address'),'"','') as ad_full_address9,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[8].last_update_dt'),'"','') as ad_last_update_dt9,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[9].source'),'"','') as ad_source10,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[9].type'),'"','') as ad_typ10,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[9].full_address'),'"','') as ad_full_address10,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[9].last_update_dt'),'"','') as ad_last_update_dt10,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[10].source'),'"','') as ad_source11,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[10].type'),'"','') as ad_typ11,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[10].full_address'),'"','') as ad_full_address11,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[10].last_update_dt'),'"','') as ad_last_update_dt11,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[11].source'),'"','') as ad_source12,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[11].type'),'"','') as ad_typ12,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[11].full_address'),'"','') as ad_full_address12,
replace(json_extract(`axp-lumi`.dw.decrypt_sde('NGBD-SDE-Address',bus_full_ad),'$[11].last_update_dt'),'"','') as ad_last_update_dt12,
prev_iclic_id,
json_value(prev_iclic_id,"$[1].iclic_id") as prev_iclic,
json_value(prev_iclic_id,"$[0].update_dt") as last_upd_dt,
clnt_org_id,
corp_id,
iclic_mtch_cd,
iclic_glbl_head_id,
iclic_ult_id,
lgl_ent_iclic_id,
parnt_dun_no,
hq_dun_no,
dom_ult_dun_no,
ult_dun_no,
hq_parnt_nm,
dom_ult_nm,
glbl_ult_nm,
acct_sta_cd,
bus_acct_eff_dt,
bus_acct_canc_dt,
auth_offcr1_nat_id,
auth_offcr1_nat_id_type,
cg_id,
corp_hier_cd,
cust_xref_id,
dun_no,
prev_oblgr_id,
rgn_loc,
canc_rsn_cd,
mkt_cd_no,
dnb_mtch_grade,
dnb_mtch_prfl,
duns_cert_dt,
sor_mkt_cd
from `axp-lumi.dw.cust360_commercial`
where {selected_value} in ({formatted_list})
""")

query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

df1 = query_job.to_dataframe()
row_count = len(df1)
print(f"Number of rows: {row_count}")

'000236606885406','000268732174137'
Number of rows: 2


## Data Cleaning for Commercial Linkage Assesment -

In [7]:
import pandas as pd
import json
# --- Step 1. Ensure lists of dicts ---
df1["bus_nm"] = df1["bus_nm"].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
df1["bus_ad"] = df1["bus_ad"].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
# --- Step 2. Compact helper for addresses ---
def build_address(addr):
    return ", ".join([addr.get(k, "") for k in [
        "addr_line1","addr_line2","addr_line3","addr_line4",
        "addr_line5","addr_line6","city","state","country","postal_code"
    ] if addr.get(k)])
# --- Step 3. Extract in one pass (names + addresses) ---
def extract_all(row):
    legal_name, dba_name, home_addr,emboss_employer, business_addr = None, None, None, None, None
    # Handle bus_nm
    for nm in row["bus_nm"] or []:
        if nm.get("type") in ["legal_name", "LEGAL"] and not legal_name:
            legal_name = nm.get("bus_name")
        if nm.get("type") in ["DBA", "dba_nm"] and not dba_name:
            dba_name = nm.get("bus_name")
        if nm.get("type") in ["EMBOSS", "emboss name", "EMPLOYER", "employer name"] and not emboss_employer:
            emboss_employer = nm.get("bus_name")
            
    # Handle bus_ad
    for ad in row["bus_ad"] or []:
        if ad.get("type") == "HOME" and not home_addr:
            home_addr = build_address(ad)
        if ad.get("type") in ["BUSINESS", "Business"] and not business_addr:
            business_addr = build_address(ad)
    return pd.Series([legal_name, dba_name, home_addr,emboss_employer, business_addr])
# --- Step 4. Apply once across rows ---
df1[["legal_name", "dba_name", "Home_address", "emboss_employer","Business_address"]] = df1.apply(extract_all, axis=1)
# --- Step 5. Final result ---
df_final_result = df1
row_count = len(df_final_result)
print(f"Number of rows: {row_count}")
#df_final_result

Number of rows: 2


## UDF For Saving the Data in Excel Format -

In [8]:
def save_dataframe_to_excel(pandas_df, default_filename="my_report"):
    """
    Saves a Spark DataFrame to an Excel .xlsx file with user-defined name
    and today's date appended.
    Args:
        spark_df (pyspark.sql.DataFrame): The Spark DataFrame to save.
        default_filename (str): A default base filename if the user doesn't provide one.
    """
    # --- 1. Get user input for base filename ---
    from datetime import datetime
    while True:
        user_input = input(f"Enter a base INC number for the Excel file (e.g., '{default_filename}'): ").strip()
        if user_input:
            base_filename = user_input
            break
        else:
            print("Filename cannot be empty. Please try again.")
    # --- 2. Add today's date to the filename ---
    today_date_str = datetime.now().strftime("%Y-%m-%d") # Format: YYYY-MM-DD
    excel_filename = f"{base_filename}_{today_date_str}.xlsx" # Using .xlsx for modern Excel format
    # --- 3. Collect Spark DataFrame to Pandas DataFrame ---
    print(f"\nCollecting Spark DataFrame to Pandas DataFrame. This may take time for large datasets...")
    try:
        pandas_df
        pd.set_option('display.float_format', '{:.0f}'.format)
        print(f"Successfully collected {len(pandas_df)} rows to Pandas DataFrame.")
    except Exception as e:
        print(f"Error collecting Spark DataFrame to Pandas: {e}")
        print("The DataFrame might be too large to fit in memory.")
        return
    # --- 4. Save Pandas DataFrame to Excel ---
    try:
        # Create the full path to save the file (e.g., in the current working directory)
        #custom_path = r"Z:\Nucleus\Prashant Saraswat"
        #output_path = os.path.join(custom_path, excel_filename)
        # Using pandas to_excel method
        Current = os.getcwd()
        output_path = os.path.join(Current, excel_filename)
        pandas_df.to_excel(output_path, index=False) # index=False prevents writing the Pandas DataFrame index as a column
        print(f"\nDataFrame successfully saved to Excel: '{output_path}'")
    except Exception as e:
        print(f"Error saving Pandas DataFrame to Excel: {e}")

## Please Provide the File Name -

In [9]:
save_dataframe_to_excel(df_final_result, default_filename="user_data_report")

Enter a base INC number for the Excel file (e.g., 'user_data_report'):  Test



Successfully collected 2 rows to Pandas DataFrame.

DataFrame successfully saved to Excel: '/home/jupyter/axp-us-psara6/Test_2026-01-15.xlsx'


In [10]:
df_final_result["cm15"] = pd.to_numeric(df_final_result["cm15"], errors="coerce").astype("Int64")
new_df = df_final_result[['cm15','iclic_id','cg_id','se10','bu_in','tax_id', 'cert_dun_no','legal_name','dba_name','emboss_employer','Business_address','Home_address']]

## Hard Match Linkage Assesment
## Please Provide the Record_Identifier/Field_Name - 

In [11]:
def match_columns_from_master_record(df, record_id, id_column, columns_to_match):
    """
    Matches specified columns in a DataFrame against a "master" record.

    Args:
        df (pd.DataFrame): The input DataFrame.
        record_id: The value in the id_column that identifies the master record.
        id_column (str): The name of the column used to identify records (e.g., 'record_id').
        columns_to_match (list): A list of column names to compare against the master record.

    Returns:
        pd.DataFrame: The DataFrame with individual match columns and a final boolean
                      column 'is_match' indicating if the record matches the master record's
                      values for all specified columns.
    """
    
    df = df.copy()
    # 1. Input validation to ensure the master record exists
    if record_id not in df[id_column].values:
        print(f"Error: Record with ID '{record_id}' not found in column '{id_column}'.")
        return df

    # 2. Identify the master record and extract its values for the specified columns
    master_record = df[df[id_column] == record_id]

    if master_record.empty:
        print(f"Error: Master record with ID '{record_id}' is empty.")
        return df

    # 3. Create a master boolean mask to compare all records
    # We initialize a mask of all True values
    final_match_mask = pd.Series(True, index=df.index)

    # 4. Iterate through the columns to match and create individual match columns
    for col in columns_to_match:
        # Check if the column exists in the DataFrame
        if col not in df.columns:
            print(f"Warning: Column '{col}' not found in the DataFrame. Skipping.")
            continue

        # Get the master record's value for the current column
        master_value = master_record[col].iloc[0]

        # Create a new boolean column for this specific match
        individual_match_column_name = f"{col}_match"
        df.loc[:, individual_match_column_name] = (df[col] == master_value)

        # Update the final match mask by combining it with the individual match column
        final_match_mask = final_match_mask & df[individual_match_column_name]

    # 5. Add the 'is_match' column to the DataFrame
    df.loc[:, "is_match"] = final_match_mask
    
    return df

# --- Get user input for master record details ---
# The input is read as a string, so we need to convert it to an integer for matching.
try:
    master_id = int(input("Enter the master record: "))
except ValueError:
    print("Invalid input. Please enter a number for the transaction ID.")
    master_id = 103 # Default to 103 if input is invalid

id_column = input("Enter the name of the ID column (e.g., 'transaction_id'): ")
# --- END NEW ---

# Assume the columns we want to match are 'product_category' and 'customer_country'.
columns_to_check = ['tax_id', 'cert_dun_no','legal_name','dba_name','emboss_employer','Business_address','Home_address']

# Call the function
matched_df = match_columns_from_master_record(
    df=new_df,
    record_id=master_id,
    id_column=id_column,
    columns_to_match=columns_to_check
)
pd.set_option('display.float_format', '{:.0f}'.format)
print("\nDataFrame with match columns:")
display(matched_df)

Enter the master record:  376011856655009
Enter the name of the ID column (e.g., 'transaction_id'):  cm15



DataFrame with match columns:


,cm15,iclic_id,cg_id,se10,bu_in,tax_id,cert_dun_no,legal_name,dba_name,emboss_employer,Business_address,Home_address,tax_id_match,cert_dun_no_match,legal_name_match,dba_name_match,emboss_employer_match,Business_address_match,Home_address_match,is_match
0,376011856655009,000268732174137,None,<NA>,O,23481084076,742373395,None,None,THE TRUSTEE FOR THE JM CONSULTANCY SERVI,None,None,True,True,False,False,True,False,False,False
1,340091130811001,000236606885406,None,<NA>,O,64660784369,750699242,None,None,GRAYSTONE PRIVATE SC PTY LTD,None,None,False,False,False,False,False,False,False,False


In [12]:
new_df = df_final_result[['cm15','iclic_id','cg_id','se10','bu_in','tax_id', 'cert_dun_no','legal_name','dba_name','emboss_employer','Business_address','Home_address']]

## Linkage Assesment Using Jaro Winkler - 

In [13]:
import pandas as pd
import jellyfish

def match_columns_from_master_record(df, record_id, id_column, columns_to_match): 
    """
    Matches specified columns in a DataFrame against a "master" record using Jaro-Winkler similarity.
    Produces similarity columns (0–100%) and a final average similarity_score.
    """

    # Always work on a copy to avoid SettingWithCopyWarning
    df = df.copy()

    # 1. Input validation
    if record_id not in df[id_column].values:
        print(f"Error: Record with ID '{record_id}' not found in column '{id_column}'.")
        return df

    # 2. Get master record
    master_record = df[df[id_column] == record_id]
    if master_record.empty:
        print(f"Error: Master record with ID '{record_id}' is empty.")
        return df

    # 3. Compare columns using Jaro-Winkler
    for col in columns_to_match:
        if col not in df.columns:
            print(f"Warning: Column '{col}' not found in the DataFrame. Skipping.")
            continue

        master_value = str(master_record[col].iloc[0]) if pd.notna(master_record[col].iloc[0]) else ""
        similarity_col_name = f"{col}_similarity"

        # Assign similarity scores (percentage)
        df[similarity_col_name] = df[col].fillna("").astype(str).apply(
            lambda x: jellyfish.jaro_winkler_similarity(str(x), master_value) * 100
        )
        
        df[similarity_col_name] = df[similarity_col_name].round().astype("Int64")


    # 4. Add final similarity score (average of all similarity columns)
    sim_cols = [c for c in df.columns if c.endswith("_similarity")]
    if sim_cols:
        df["similarity_score"] = (
        df[sim_cols]
        .replace(0, pd.NA)   # 👈 ignore zero similarity values
        .mean(axis=1)
        .round()
        .astype("Int64")
         )

    return df

#try:
 #   master_id = int(input("Enter the master record: "))
#except ValueError:
 #   print("Invalid input. Please enter a number for the transaction ID.")
  #  master_id = 103 # Default to 103 if input is invalid

#id_column = input("Enter the name of the ID column (e.g., 'transaction_id'): ")
# --- END NEW ---

# Assume the columns we want to match are 'product_category' and 'customer_country'.
columns_to_check = ['tax_id', 'cert_dun_no','legal_name','dba_name','emboss_employer','Business_address','Home_address']

# Call the function
matched_df = match_columns_from_master_record(
    df=new_df,
    record_id=master_id,
    id_column=id_column,
    columns_to_match=columns_to_check
)
pd.set_option('display.float_format', '{:.0f}'.format)
print("\nDataFrame with match columns:")
row_count = len(matched_df)
print(f"Number of rows: {row_count}")
#display(matched_df)


DataFrame with match columns:
Number of rows: 2


In [14]:
def highlight_rows(row):
    if pd.notna(row[id_column]) and str(row[id_column]) == str(master_id):
        return ['color: red']*len(row)
    else:
        return ['']*len(row)

def highlight_rows_o(row):
    if pd.notna(row['bu_in']) and row['bu_in'] == 'O':
        return ['background-color: #90EE90']*len(row)
    else:
        return ['']*len(row)

matched_df = matched_df.sort_values(by='bu_in', ascending=False)

matched_df.style.apply(highlight_rows_o, axis=1).apply(highlight_rows, axis=1)


,cm15,iclic_id,cg_id,se10,bu_in,tax_id,cert_dun_no,legal_name,dba_name,emboss_employer,Business_address,Home_address,tax_id_similarity,cert_dun_no_similarity,legal_name_similarity,dba_name_similarity,emboss_employer_similarity,Business_address_similarity,Home_address_similarity,similarity_score
0,376011856655009,000268732174137,None,,O,23481084076,742373395,None,None,THE TRUSTEE FOR THE JM CONSULTANCY SERVI,None,None,100,100,0,0,100,0,0,100
1,340091130811001,000236606885406,None,,O,64660784369,750699242,None,None,GRAYSTONE PRIVATE SC PTY LTD,None,None,59,48,0,0,59,0,0,55


In [15]:
# Ensure required columns exist and are properly typed
if {"similarity_score", "bu_in"}.issubset(matched_df.columns):
    # Convert similarity_score to numeric (in case it's Int64 or string)
    matched_df["similarity_score"] = pd.to_numeric(matched_df["similarity_score"], errors="coerce")

    # Apply both conditions:
    # 1️⃣ similarity_score >= 80
    # 2️⃣ bu_in is not 'E'
    # 3️⃣ similarity_score != 80
    filtered_df = matched_df[(matched_df["similarity_score"] >= 80) & (matched_df["bu_in"] != "E") & (matched_df["similarity_score"] != 100) ]

    if filtered_df.empty:
        print("No similar records exist — Delinking will be required.")
    else:
        print("Similar records exist — Linkage is correct.")
else:
    print("Missing one of the required columns: 'similarity_score' or 'bu_in'.")


No similar records exist — Delinking will be required.


## RCA - 

In [16]:
if {"similarity_score", "bu_in"}.issubset(matched_df.columns):
    # Convert similarity_score to numeric (in case it's Int64 or string)
    matched_df["similarity_score"] = pd.to_numeric(matched_df["similarity_score"], errors="coerce")

    # Apply both conditions:
    # 1️⃣ similarity_score >= 80
    # 2️⃣ bu_in is not 'E'
    # 3️⃣ similarity_score != 80
    filtered_df = matched_df[(matched_df["similarity_score"] >= 80) & (matched_df["bu_in"] != "E") & (matched_df["similarity_score"] != 100) ]
    # Run only if linkage was incorrect (no valid rows)
    if filtered_df.empty:
        print("No valid linkage rows found — checking master record for incorrect linkage reason...")
    
        # Define mapping of (mtch_cd, mtch_cd_mi) → print message
        pair_to_msg = {
            ("3", "9"): "Linkage incorrect due to Arbitration",
            ("",  "9"): "Linkage incorrect due to Mannual Changes",
            ("3", "G"): "Linkage incorrect due to Internal Matching",
            ("3", "A"): "Linkage incorrect due to DnB"
        }
    
        # Select only the master record from df_final_result
        master_row = df_final_result[df_final_result[id_column] == master_id].copy()
    
        if master_row.empty:
            print(f"No master record found for ID '{master_id}'.")
        else:
            # Ensure columns exist
            if {"mtch_cd", "mtch_cd_mi"}.issubset(master_row.columns):
                # Normalize for safe string comparison
                master_row["mtch_cd"] = master_row["mtch_cd"].astype(str).fillna("").replace("nan", "")
                master_row["mtch_cd_mi"] = master_row["mtch_cd_mi"].astype(str).fillna("").replace("nan", "")
    
                # Extract the pair for master record
                mtch_cd_val = master_row["mtch_cd"].iloc[0]
                mtch_cd_mi_val = master_row["mtch_cd_mi"].iloc[0]
    
                # Check if this pair matches any invalid mapping
                message = pair_to_msg.get((mtch_cd_val, mtch_cd_mi_val))
                if message:
                    print(f"{message} (mtch_cd = '{mtch_cd_val}', mtch_cd_mi = '{mtch_cd_mi_val}')")
                else:
                    print("Master record linkage does not match any invalid mtch_cd / mtch_cd_mi combination.")
            else:
                print("Columns 'mtch_cd' or 'mtch_cd_mi' not found in df_final_result.")
    else:
        print("Linkage is correct — skipping mtch_cd / mtch_cd_mi check.")
        
    def get_linkage_resolution(df_final_result, id_column, master_id, filtered_df):
        """
        Returns the resolution string for the master record based on mtch_cd and mtch_cd_mi values.
        Runs only if linkage is incorrect (filtered_df is empty).
        """
    
        if filtered_df.empty:
            # Mapping of (mtch_cd, mtch_cd_mi) → Resolution
            pair_to_resolution = {
                ("3", "9"): "May require Manual Changes as matched due to arbitration",
                ("",  "9"): "No Changes could be done in this case",
                ("3", "G"): "Should be sent for Internal Rematching",
                ("3", "A"): "Should be sent for External Rematching/Check the records at DnB portal"
            }
    
            # Select only the master record and make a copy to avoid SettingWithCopyWarning
            master_row = df_final_result[df_final_result[id_column] == master_id].copy()
    
            if master_row.empty:
                return f"No master record found for ID '{master_id}'."
    
            if {"mtch_cd", "mtch_cd_mi"}.issubset(master_row.columns):
                # Normalize values for comparison
                mtch_cd_val = str(master_row["mtch_cd"].iloc[0]) if pd.notna(master_row["mtch_cd"].iloc[0]) else ""
                mtch_cd_mi_val = str(master_row["mtch_cd_mi"].iloc[0]) if pd.notna(master_row["mtch_cd_mi"].iloc[0]) else ""
    
                # Normalize 'nan' literals
                if mtch_cd_val.lower() == "nan":
                    mtch_cd_val = ""
                if mtch_cd_mi_val.lower() == "nan":
                    mtch_cd_mi_val = ""
    
                # Return the matching resolution (or fallback)
                resolution = pair_to_resolution.get((mtch_cd_val, mtch_cd_mi_val))
                if resolution:
                    return resolution
                else:
                    return (
                        f"No predefined resolution found for mtch_cd = '{mtch_cd_val}', "
                        f"mtch_cd_mi = '{mtch_cd_mi_val}'."
                    )
            else:
                return "Columns 'mtch_cd' or 'mtch_cd_mi' not found in df_final_result."
        else:
            return "Linkage is correct — no resolution lookup required."
else:
    print("Missing one of the required columns: 'similarity_score' or 'bu_in'.")


No valid linkage rows found — checking master record for incorrect linkage reason...
Linkage incorrect due to DnB (mtch_cd = '3', mtch_cd_mi = 'A')


## Resolution - 

In [17]:
resolution_message = get_linkage_resolution(df_final_result, id_column, master_id, filtered_df)
print(resolution_message)

Should be sent for External Rematching/Check the records at DnB portal


## C360 API - 

In [18]:
pip install requests-ntlm

Looking in indexes: https://gcprepo.aexp.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [19]:
import requests
import json
# --- 1. Define the API Endpoint and Parameters ---
# API for daily weather forecast (free and public)
API_URL = "https://api.open-meteo.com/v1/forecast"
# Parameters for New York City (Latitude/Longitude)
# We request the 'daily' forecast and specifically the 'temperature_2m_max'
parameters = {
    "latitude": 40.7128,    # New York City Latitude
    "longitude": -74.0060, # New York City Longitude
    "daily": "temperature_2m_max",
    "timezone": "America/New_York",
    "forecast_days": 7      # Request 7 days of data
}
def fetch_weather_data(url, params):
    """
    Makes the API request and returns the parsed JSON data.
    """
    print(f"Attempting to fetch data for coordinates: {params['latitude']}, {params['longitude']}")
    try:
        # Send a GET request with parameters
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status() # Check for HTTP errors
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f":x: Error fetching data: {e}")
        return None
# --- Main Execution ---
weather_data = fetch_weather_data(API_URL, parameters)
if weather_data:
    # 2. Extract and display the desired output
    print("\n:white_check_mark: API Request Successful!")
    # Extract the daily time stamps and max temperatures
    daily_times = weather_data.get('daily', {}).get('time', [])
    max_temps = weather_data.get('daily', {}).get('temperature_2m_max', [])
    unit = weather_data.get('daily_units', {}).get('temperature_2m_max', '°C')
    print("\n--- 7-Day Max Temperature Forecast for NYC ---")
    # Pair the date and temperature together for clean output
    for date, temp in zip(daily_times, max_temps):
        print(f":date: {date}: {temp}{unit}")
else:
    print("\n:x: Failed to retrieve weather forecast data.")

Attempting to fetch data for coordinates: 40.7128, -74.006
:x: Error fetching data: HTTPSConnectionPool(host='api.open-meteo.com', port=443): Max retries exceeded with url: /v1/forecast?latitude=40.7128&longitude=-74.006&daily=temperature_2m_max&timezone=America%2FNew_York&forecast_days=7 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f6f143c8f10>: Failed to establish a new connection: [Errno 101] Network is unreachable'))

:x: Failed to retrieve weather forecast data.


## Test - 

In [20]:
# Run only if linkage was incorrect (no valid rows)
if filtered_df.empty:
    print("No valid linkage rows found — checking master record for incorrect linkage reason...")

    # Define mapping of (mtch_cd, mtch_cd_mi) → print message
    pair_to_msg = {
        ("3", "9"): "Linkage incorrect due to Arbitration",
        ("",  "9"): "Linkage incorrect due to Mannual Changes",
        ("3", "G"): "Linkage incorrect due to Internal Matching",
        ("3", "A"): "Linkage incorrect due to DnB"
    }

    # Select only the master record from df_final_result
    master_row = df_final_result[df_final_result[id_column] == master_id].copy()

    if master_row.empty:
        print(f"No master record found for ID '{master_id}'.")
    else:
        # Ensure columns exist
        if {"mtch_cd", "mtch_cd_mi"}.issubset(master_row.columns):
            # Normalize for safe string comparison
            master_row["mtch_cd"] = master_row["mtch_cd"].astype(str).fillna("").replace("nan", "")
            master_row["mtch_cd_mi"] = master_row["mtch_cd_mi"].astype(str).fillna("").replace("nan", "")

            # Extract the pair for master record
            mtch_cd_val = master_row["mtch_cd"].iloc[0]
            mtch_cd_mi_val = master_row["mtch_cd_mi"].iloc[0]

            # Check if this pair matches any invalid mapping
            message = pair_to_msg.get((mtch_cd_val, mtch_cd_mi_val))
            if message:
                print(f"{message} (mtch_cd = '{mtch_cd_val}', mtch_cd_mi = '{mtch_cd_mi_val}')")
            else:
                print("Master record linkage does not match any invalid mtch_cd / mtch_cd_mi combination.")
        else:
            print("Columns 'mtch_cd' or 'mtch_cd_mi' not found in df_final_result.")
else:
    print("Linkage is correct — skipping mtch_cd / mtch_cd_mi check.")
    
def get_linkage_resolution(df_final_result, id_column, master_id, filtered_df):
    """
    Returns the resolution string for the master record based on mtch_cd and mtch_cd_mi values.
    Runs only if linkage is incorrect (filtered_df is empty).
    """

    if filtered_df.empty:
        # Mapping of (mtch_cd, mtch_cd_mi) → Resolution
        pair_to_resolution = {
            ("3", "9"): "May require Manual Changes as matched due to arbitration",
            ("",  "9"): "No Changes could be done in this case",
            ("3", "G"): "Should be sent for Internal Rematching",
            ("3", "A"): "Should be sent for External Rematching/Check the records at DnB portal"
        }

        # Select only the master record and make a copy to avoid SettingWithCopyWarning
        master_row = df_final_result[df_final_result[id_column] == master_id].copy()

        if master_row.empty:
            return f"No master record found for ID '{master_id}'."

        if {"mtch_cd", "mtch_cd_mi"}.issubset(master_row.columns):
            # Normalize values for comparison
            mtch_cd_val = str(master_row["mtch_cd"].iloc[0]) if pd.notna(master_row["mtch_cd"].iloc[0]) else ""
            mtch_cd_mi_val = str(master_row["mtch_cd_mi"].iloc[0]) if pd.notna(master_row["mtch_cd_mi"].iloc[0]) else ""

            # Normalize 'nan' literals
            if mtch_cd_val.lower() == "nan":
                mtch_cd_val = ""
            if mtch_cd_mi_val.lower() == "nan":
                mtch_cd_mi_val = ""

            # Return the matching resolution (or fallback)
            resolution = pair_to_resolution.get((mtch_cd_val, mtch_cd_mi_val))
            if resolution:
                return resolution
            else:
                return (
                    f"No predefined resolution found for mtch_cd = '{mtch_cd_val}', "
                    f"mtch_cd_mi = '{mtch_cd_mi_val}'."
                )
        else:
            return "Columns 'mtch_cd' or 'mtch_cd_mi' not found in df_final_result."
    else:
        return "Linkage is correct — no resolution lookup required."

No valid linkage rows found — checking master record for incorrect linkage reason...
Linkage incorrect due to DnB (mtch_cd = '3', mtch_cd_mi = 'A')
